# Comments to the following code
> - #### Parts 1 and 2 can be run separately
> - #### Raw data: Excel sheet can be obtained from https://doi.org/10.1021/acs.est.8b01452
> - #### Terms `exchange` and `input activity` are used interchangeably
> - #### Only take ecoinvent inventories for the paper
> - #### Demand would be one of the sectors in the 

# ------------------------------------------ Part 1 ------------------------------------------

In [1]:
import pandas as pd
import numpy as np
from copy import copy, deepcopy
import os, json
import re
import brightway2 as bw

# Local files
from utils_consumption_db import *

# Convert data to brightway database format -> Additional functions

In [2]:
def append_ecoinvent_exchange(df, df_ind_j, ConversionDem2FU):
    '''
    Extract information about one input activity, eg name, unit, location, etc and append it to the dataframe df.
    '''    
    # Extract the activity number
    k = int(''.join(c for c in df_ind_j.index[0] if c.isdigit()))
    # Extract information about activity and save it
    input_act_str = df_ind_j['DB Act ' + str(k)]
    input_act_db_code = df_ind_j['Activity ' + str(k)]
    
    # Find this input activity in brightway databases
    db_name = input_act_db_code.split("'")[1]
    
    # Only include ecoinvent exchanges
    dbs_no_list = ['agribalyse', 'exiobase', 'heia']
    for db_no in dbs_no_list:
        if db_no in db_name.lower():
            return df
        
    code = input_act_db_code.split("'")[3]
    input_act_db_code_tuple = (db_name, code)
    
    # Compute amount
    input_act_amount = df_ind_j['On ' + str(k)] \
                     * df_ind_j['Amount Act ' + str(k)] \
                     * df_ind_j['CFL Act ' + str(k)] \
                     * ConversionDem2FU
    
    try:
        # Find activity in the ecoinvent 3.3 cutoff using bw functionality
        current_project = deepcopy(bw.projects.current)
        bw.projects.set_current('ecoinvent 3.3 cutoff') # Temporarily switch to ecoinvent 3.3 project
        act_bw = bw.get_activity(input_act_db_code_tuple)
        bw.projects.set_current(current_project)
        input_act_values_dict = create_input_act_dict(act_bw, input_act_amount)
        
    except:
        # If bw.get_activity does not work for whichever reason, fill info manually
        input_act_values_dict = bw_get_activity_info_manually(input_act_str, db_name, input_act_amount)
        
    # Add exchange to the dataframe with database in brightway format
    df = append_exchanges_in_correct_columns(df, input_act_values_dict)
    
    return df

# Convert data to brightway database format -> main code
calls all the functions used above

In [3]:
with open('global_settings.json', 'rb') as f:
    settings = json.load(f)
which_pc = settings['which_pc']

In [4]:
%%time
# 0. Set relevant paths
if which_pc == 'local':
    path_habe = '/Users/akim/Documents/LCA_files/HABE_2017/'
    ei33_path = '/Users/akim/Documents/LCA_files/ecoinvent 3.3 cutoff/datasets'  
elif which_pc == 'merlin':
    path_habe = '/data/user/kim_a/LCA_files/HABE_2017'
    ei33_path = '/data/user/kim_a/LCA_files/ecoinvent_33_cutoff/datasets'  
    
# 1. Create project with ecoinvent 3.3 cutoff only. 
# This is needed because Andi's activities were given as ecoinvent 3.3 cutoff codes
create_ecoinvent_33_project(ei33_path)

# Start brightway project that already contains databases
project = 'GSA for paper'
bw.projects.set_current(project)

# Create dataframe that will be our consumption database after we add activities and exchanges from the raw file
df_bw = create_df_bw(CONSUMPTION_DB_NAME)

# Read data from Andi's consumption database
path = 'data/es8b01452_si_002.xlsx'
sheet_name = 'Overview & LCA-Modeling'
df_raw = pd.read_excel(path, sheet_name = sheet_name, header=2)

# Read data from HABE
year = '091011' #121314 or 151617
code_unit = get_units_habe(path_habe, year)

# Add ON columns
df = complete_columns(df_raw)

act_indices = df_raw.index[df_raw['ConversionDem2FU'].notna()].tolist() # indices of all activities

sector_codes = ['a51', 'a52'] # Food and non-alcoholic beverages
for ind in act_indices:
    # For each row
    df_ind = df_raw.iloc[ind]
    df_ind = df_ind[df_ind.notna()]
    
    if df_ind['Variable code'][:3] in sector_codes:
        # Add activity
        df_bw, df_act = append_activity(df_bw, df_ind[:N_ACT_RELEVANT], code_unit) # only pass columns relevant to this function
        # Add exchanges
        df_bw = append_exchanges(df_bw, df_ind, df_act, append_one_exchange=append_ecoinvent_exchange)

Creating default biosphere



Writing activities to SQLite3 database:


Applying strategy: normalize_units
Applying strategy: drop_unspecified_subcategories
Applied 2 strategies in 0.00 seconds


0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:00


Title: Writing activities to SQLite3 database:
  Started: 06/10/2020 13:09:19
  Finished: 06/10/2020 13:09:20
  Total time elapsed: 00:00:00
  CPU %: 49.20
  Memory %: 0.04
Created database: biosphere3
Creating default LCIA methods

Applying strategy: normalize_units
Applying strategy: set_biosphere_type
Applying strategy: drop_unspecified_subcategories
Applying strategy: link_iterable_by_fields
Applied 4 strategies in 1.25 seconds
Wrote 850 LCIA methods with 219059 characterization factors
Creating core data migrations

Extracting XML data from 13831 datasets
Extracted 13831 datasets in 65.39 seconds
Applying strategy: normalize_units
Applying strategy: update_ecoinvent_locations
Applying strategy: remove_zero_amount_coproducts
Applying strategy: remove_zero_amount_inputs_with_no_activity
Applying strategy: remove_unnamed_parameters
Applying strategy: es2_assign_only_product_with_amount_as_reference_product
Applying strategy: assign_single_product_as_activity
Applying strategy: create

Writing activities to SQLite3 database:


13831 datasets
501148 exchanges
0 unlinked exchanges
  


0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:58


Title: Writing activities to SQLite3 database:
  Started: 06/10/2020 13:11:49
  Finished: 06/10/2020 13:12:48
  Total time elapsed: 00:00:58
  CPU %: 47.70
  Memory %: 0.32
Created database: ecoinvent 3.3 cutoff
CPU times: user 2min 43s, sys: 21 s, total: 3min 4s
Wall time: 7min 4s


In [5]:
# Write the dataframe to excel file
write_dir_name = 'write_files'
if not os.path.exists(write_dir_name):
    os.mkdir(write_dir_name)
db_bw_path = write_dir_name + '/' + 'consumption_db_paper.xlsx'
df_bw.to_excel(db_bw_path, index=False, header=False)

# ------------------------------------------ Part 2 ------------------------------------------

In [6]:
import pandas as pd
import numpy as np
import brightway2 as bw
import string
from copy import copy, deepcopy

# Local files
from utils import *
from utils_consumption_db import *

# Constants

In [7]:
DB_COLUMN = 'F'
CONSUMPTION_DB_NAME = 'CH consumption 1.0'

# Replace names of old databases with the new ones in the consumption database excel file

In [8]:
def update_all_db(df):
    '''
    Update all databases in the consumption database
    '''
    db_old_list = ['ecoinvent 3.3 cutoff']
    db_new_list = ['ecoinvent 3.6 cutoff']
    
    assert len(db_old_list) == len(db_new_list)
    
    for i in range(len(db_old_list)):
        df = replace_one_db(df, db_old_list[i], db_new_list[i])
        
    return df

In [9]:
# Main code
project = 'GSA for paper'
bw.projects.set_current(project)

# Read consumption database
path = 'write_files/consumption_db_paper.xlsx'
df = pd.read_excel(path, header = None)
df.columns = list(string.ascii_uppercase[:len(df.columns)])
# 
# Replace
df = update_all_db(df)
path_new_db = 'write_files/consumption_db_paper_updated.xlsx'
df.to_excel(path_new_db, index=False, header=False)

# Import consumption database linked to older versions of other databases

# 1. Ecoinvent 3.6

### TODO Chris -> please check migrations

In [10]:
if CONSUMPTION_DB_NAME in bw.databases:
    del bw.databases[CONSUMPTION_DB_NAME]

In [11]:
co = bw.ExcelImporter(path_new_db)
co.apply_strategies()
co.match_database('ecoinvent 3.6 cutoff', fields=('name', 'reference product', 'unit','location','categories'))
co.match_database('ecoinvent 3.6 cutoff', fields=('name', 'unit','location','categories'))
co.statistics()

Extracted 1 worksheets in 0.06 seconds
Applying strategy: csv_restore_tuples
Applying strategy: csv_restore_booleans
Applying strategy: csv_numerize
Applying strategy: csv_drop_unknown
Applying strategy: csv_add_missing_exchanges_section
Applying strategy: normalize_units
Applying strategy: normalize_biosphere_categories
Applying strategy: normalize_biosphere_names
Applying strategy: strip_biosphere_exc_locations
Applying strategy: set_code_by_activity_hash
Applying strategy: link_iterable_by_fields
Applying strategy: assign_only_product_as_production
Applying strategy: link_technosphere_by_activity_hash
Applying strategy: drop_falsey_uncertainty_fields_but_keep_zeros
Applying strategy: convert_uncertainty_types_to_integers
Applying strategy: convert_activity_parameters_to_list
Applied 16 strategies in 0.54 seconds
Applying strategy: link_iterable_by_fields
Applying strategy: link_iterable_by_fields
89 datasets
658 exchanges
95 unlinked exchanges
  Type technosphere: 11 unique unlinked

(89, 658, 95)

In [12]:
# Define a migration for two particular activities that can only be hardcoded
ecoinvent36_change_names_data = {
    'fields': ['name', ],
    'data': [
        (
            ['steam production in chemical industry'], 
            {
                'name': 'steam production, in chemical industry',
                'reference product': 'steam, in chemical industry',
                'unit': 'kilogram',
                'multiplier': 1/2.75, # see comment on this activity in ecoinvent
            }
        ),
        (
            ['market for green bell pepper'],
            {
                'name': 'market for bell pepper',
                'reference product': 'bell pepper',
            }
        ),
    ]
}

bw.Migration("ecoinvent36-change-names").write(
    ecoinvent36_change_names_data,
    description="Change names of some activities"
)

In [13]:
# Define a migration for rice production and specific locations
# These locations have only non-basmati rice production
ecoinvent36_rice_production_data = {
    'fields': ['name', 'location'],
    'data': [
        (
            ['rice production', 'US'],
            {
                'name': 'rice production, non-basmati',
                'reference product': 'rice, non-basmati'
            }
        ),
        (
            ['rice production', 'CN'],
            {
                'name': 'rice production, non-basmati',
                'reference product': 'rice, non-basmati'
            }
        ),
    ]
}

bw.Migration("ecoinvent36-rice-production").write(
    ecoinvent36_rice_production_data,
    description="Change names of some activities"
)

In [14]:
co.migrate('ecoinvent36-change-names')
co.migrate("ecoinvent36-rice-production")
co.match_database('ecoinvent 3.6 cutoff', fields=('name','reference product', 'unit','location','categories'))
co.statistics()

Applying strategy: migrate_datasets
Applying strategy: migrate_exchanges
Applying strategy: migrate_datasets
Applying strategy: migrate_exchanges
Applying strategy: link_iterable_by_fields
89 datasets
658 exchanges
26 unlinked exchanges
  Type technosphere: 7 unique unlinked exchanges


(89, 658, 26)

The rest of the unlinked exchanges are not uniquely defined in ecoinvent 3.6 -> 1-to-multiple mapping. <br>
For example 'rice production' is now divided into basmati and non-basmati rice. <br>
Hence, we split them based on their shares in the production volumes.

In [15]:
# Manually choose which ecoinvent 3.6 exchanges should be taken for each unlinked exchange
ei36 = bw.Database('ecoinvent 3.6 cutoff')
mapping = [
    {('market for rice', 'GLO'): 
        [act['code'] for act in ei36 if  'market for rice' in act['name'] 
                                     and act['location']=='GLO'
                                     and 'seed' not in act['name']]},
    
    {('rice production', 'RoW'): 
        [act['code'] for act in ei36 if  'rice production' in act['name'] 
                                     and act['location']=='RoW'
                                     and 'straw' not in act['reference product']]},
    
    {('rice production', 'IN'): 
        [act['code'] for act in ei36 if  'rice production' in act['name'] 
                                     and act['location']=='IN'
                                     and 'straw' not in act['reference product']]},
    
    {('market for wheat grain', 'GLO'): 
        [act['code'] for act in ei36 if  'market for wheat grain' in act['name'] 
                                     and 'feed' not in act['name']]},
    
    {('market for maize grain', 'GLO'): 
        [act['code'] for act in ei36 if  'market for maize grain' in act['name'] 
                                     and 'feed' not in act['name']]},
    
    {('market for mandarin', 'GLO'): 
        [act['code'] for act in ei36 if 'market for mandarin' in act['name']]},
    
    {('market for soybean', 'GLO'): 
        [act['code'] for act in ei36 if 'market for soybean' in act['name'] 
                             and all([_ not in act['name'] for _ in ['meal','beverage','seed','feed','oil']] )]},
]

In [16]:
ei_name = 'ecoinvent 3.6 cutoff'

In [17]:
co = modify_exchanges(co, mapping, 'ecoinvent 3.6 cutoff')
co.statistics()

Applying strategy: link_iterable_by_fields
89 datasets
719 exchanges
0 unlinked exchanges
  


(89, 719, 0)

In [18]:
if CONSUMPTION_DB_NAME in bw.databases:
    print(CONSUMPTION_DB_NAME + " database already present!!! No import is needed")
else:
    co.write_database()

Writing activities to SQLite3 database:
0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:02


Title: Writing activities to SQLite3 database:
  Started: 06/10/2020 13:36:24
  Finished: 06/10/2020 13:36:27
  Total time elapsed: 00:00:02
  CPU %: 3.20
  Memory %: 0.12
Created database: CH consumption 1.0


In [19]:
bw.databases

Databases dictionary with 3 object(s):
	CH consumption 1.0
	biosphere3
	ecoinvent 3.6 cutoff

# ------------------------------------------ Part 3 ------------------------------------------

In [20]:
import brightway2 as bw
import numpy as np
import pandas as pd
import os

# Add consumption activity for all households

## 1. Extract total demand from HABE

In [21]:
with open('global_settings.json', 'rb') as f:
    settings = json.load(f)
which_pc = settings['which_pc']

In [22]:
if which_pc == 'local':
    path = '/Users/akim/Documents/LCA_files/HABE_2017/'
elif which_pc == 'merlin':
    path = '/data/user/kim_a/LCA_files/HABE_2017'
    
year = '091011' #11, 14 or 17
get_path = lambda which_file: os.path.join( path, [f for f in os.listdir(path) if year in f and which_file in f][0] )
path_beschrei = get_path('Datenbeschreibung')
path_ausgaben = get_path('Ausgaben')

In [23]:
# change codes to be consistent with consumption database and Andi's codes
project = 'GSA for paper'
bw.projects.set_current(project)
co = bw.Database('CH consumption 1.0')

ausgaben = pd.read_csv(path_ausgaben, sep='\t')

codes_co_db = sorted([act['code'] for act in co])
columns_a = ausgaben.columns.values
columns_m = [columns_a[0]]
for code_a in columns_a[1:]:
    code_m = code_a.replace('A', 'm') 
    if code_m in codes_co_db:
        columns_m.append(code_m)
    else:
        columns_m.append(code_a.lower())
        
ausgaben.columns = columns_m

In [24]:
# Compute total consumption
total_consumption = ausgaben.sum()
total_consumption = total_consumption.drop('HaushaltID')

In [25]:
# Add other useful info, eg number of households and number of people
meta = pd.read_excel(path_beschrei, sheet_name='Tabellen', skiprows=8, usecols=[0,1,3,4])
meta.columns = ['category1', 'category2', 'n_rows', 'n_cols']
meta.dropna(subset=['n_rows'], inplace=True)

# Combine some columns together
temp1 = meta[meta['category1'].notnull()][['category1', 'n_rows', 'n_cols']]
temp1.columns = ['category2', 'n_rows', 'n_cols']
temp2 = meta[meta['category2'].notnull()][['category2', 'n_rows', 'n_cols']]
meta = pd.concat([temp1, temp2])
meta.set_index('category2', inplace=True)

# Add info
total_consumption['n_households'] = meta.loc['HABE091011_Ausgaben']['n_rows']
total_consumption['n_people']     = meta.loc['HABE091011_Personen']['n_rows']

In [26]:
# Save total demand
path_demand = 'write_files/habe_totaldemands.xlsx'
total_consumption.to_excel(path_demand)

#### OPTION 1. Total demands extract directly from HABE raw files
Excel file `habe_totaldemands.xlsx` contains sums of all private households in Switzerland for all categories of the HBS. Units are the same as in the HBS (please refer to the SI-excel of Andi's ES&T-paper in order to translate the codenames). The attached vector is in "per month" quantities.

#### OPTION 2. Andi's total demands from his Swiss consumption model
Excel file `heia2_totaldemands.xlsx` contains sums of all private households in Switzerland for all categories of the HBS. Please note that the units are basically the same as in the HBS (please refer to the SI-excel of Andi's ES&T-paper in order to translate the codenames). However, the attached vector is in "per year" instead of in "per month". Furthermore, there are a couple of demands that were computed by the model itself. The codenames for these computed/imputed categories start with "mx" and the units are as follows:
* kWh per year for electricity
* MJ per year for heating
* cubic meters per year for water supply and wastewater collection
* number of waste bags per year for refuse collection

In [27]:
option = 'aggregated'
if option == 'aggregated':
#     path = 'data/heia2_totaldemands.xlsx'm # from Andi's model
#     number_households = 3518878
    path = 'write_files/habe_totaldemands.xlsx'
    df   = pd.read_excel(path)
    df.columns = ['code', 'amount']
    df.set_index('code', inplace=True)

    n_households = int(df.loc['n_households', 'amount'])
    n_people     = int(df.loc['n_people', 'amount'])
    df = df.drop(['n_households', 'n_people'])
    df = df.reset_index()
    
elif option == 'disaggregated':
    path = 'data/habe20092011_hh_prepared_imputed.csv'
    df = pd.read_csv(path, low_memory=False)
    number_households = df.shape[0]
    df = df.drop('haushaltid', axis=1).sum()
    df = df.reset_index()
    df.columns = ['code', 'amount']

# for i in range(len(df)):
#     code = df.iloc[i]['code']
#     new_code = code
#     df.at[i,'code'] = new_code

In [28]:
# Add total inputs from Andi's model as swiss consumption activity
try: co.get('ch hh food consumption').delete()
except: pass
consumption_food = co.new_activity(
    'ch hh food consumption', 
    name='ch hh food consumption', 
    location='CH', 
    unit='1 month of consumption'
)
consumption_food.save()

In [29]:
# Add production exchange for the activity `consumption`
consumption_food.new_exchange(
    input = (consumption_food['database'], consumption_food['code']),
    amount = 1,
    type = 'production'
).save()

In [30]:
codes = [act['code'] for act in co]

unlinked_codes = []
for i in range(len(df)):
    code = df.loc[i]['code']
    if code in codes:
        consumption_food.new_exchange(
            input  = (co.name, code), 
            amount = df.loc[i]['amount'], # TODO?? divide by number of months
            type   = 'technosphere'
        ).save()
    else:
        unlinked_codes.append(code)

In [31]:
# list(consumption_food.exchanges()) # per month for all households

In [32]:
len(list(consumption_food.exchanges()))

90

### Note that the number of consumption exchanges is the same as the number of activities in the database, but is a lot less than what Andi provided in his total demands.

# Add consumption activity for an average household

In [33]:
try: co.get('ch hh average food consumption').delete()
except: pass
consumption_average = consumption_food.copy(
    'ch hh average food consumption', 
    name='ch hh average food consumption'
)

In [34]:
for exc in consumption_average.exchanges():
    if exc['type'] != 'production':
        exc['amount'] /= n_households
        exc.save()

In [35]:
list(consumption_average.exchanges())

[Exchange: 1 1 month of consumption 'ch hh average food consumption' (1 month of consumption, CH, None) to 'ch hh average food consumption' (1 month of consumption, CH, None)>,
 Exchange: 2.6523185730240124 kilogram 'Rice' (kilogram, CH, None) to 'ch hh average food consumption' (1 month of consumption, CH, None)>,
 Exchange: 10.258710269091356 kilogram 'Pasta products' (kilogram, CH, None) to 'ch hh average food consumption' (1 month of consumption, CH, None)>,
 Exchange: 31.169177290477347 kilogram 'Bread' (kilogram, CH, None) to 'ch hh average food consumption' (1 month of consumption, CH, None)>,
 Exchange: 46.03483906003796 kilogram 'Bakery products (sweet and salty)' (kilogram, CH, None) to 'ch hh average food consumption' (1 month of consumption, CH, None)>,
 Exchange: 2.96203564798639 kilogram 'Sandwich' (kilogram, CH, None) to 'ch hh average food consumption' (1 month of consumption, CH, None)>,
 Exchange: 1.5632126260770978 kilogram 'Wheat flour' (kilogram, CH, None) to 'ch h

# Simple LCA

In [36]:
import brightway2 as bw

In [37]:
bw.projects.set_current('GSA for paper')
co = bw.Database('CH consumption 1.0')

In [38]:
consumption_average = co.search('average consumption')[0]

In [39]:
# demand = {consumption_average: 1}
demand = {consumption_average: 1}
method = ('IPCC 2013', 'climate change', 'GTP 100a')

In [40]:
%%time
lca = bw.LCA(demand, method)
lca.lci()
lca.lcia()
print(str(lca.score) + ' ' + bw.Method(method).metadata['unit'])

2509.0186220089945 kg CO2-Eq
CPU times: user 2 s, sys: 636 ms, total: 2.63 s
Wall time: 4.41 s


# ------------------------------------------ Part 4 ------------------------------------------

# Add categories for all activities (in English)

In [41]:
import brightway2 as bw
import numpy as np
import pandas as pd
import re

In [42]:
bw.projects.set_current('GSA for paper')

In [43]:
path = 'data/es8b01452_si_002.xlsx'
sheet_name = 'Overview & LCA-Modeling'
df_raw = pd.read_excel(path, sheet_name = sheet_name, header=2)

In [44]:
categories_col_de = 'Original name in Swiss household budget survey'
categories_col_en = 'Translated name'
categories_raw = df_raw[[categories_col_de, categories_col_en]]

In [45]:
categories = {}
for v in categories_raw.values:
    v_list_de = v[0].split(':')
    v_list_en = v[1].split(':')
    if len(v_list_de)>1 and len(v_list_de[0].split('.')) == 1:
        categories[v_list_de[0]] = v_list_en[0]
max_code_len = max({len(k) for k in categories.keys()})

In [46]:
co = bw.Database('CH consumption 1.0')

In [47]:
%%time
category_names_dict = {
    2: 'coarse',
    3: 'middle',
    4: 'fine',
}
for act in co:    
    code = re.sub(r'[a-z]+', '', act['code'], re.I)[:max_code_len]
    for i in range(2,max_code_len+1):
        try:
            category_name = 'category_' + category_names_dict[i]
            act[category_name] = categories[code[:i]]
            act.save()
        except:
            pass

CPU times: user 3.54 s, sys: 1.54 s, total: 5.08 s
Wall time: 17.7 s


In [48]:
co.random().as_dict()

{'reference product': 'Sandwich',
 'code': 'a511105',
 'location': 'CH',
 'amount': 1.0,
 'unit': 'kilogram',
 'name': 'Sandwich',
 'worksheet name': 'Sheet1',
 'database': 'CH consumption 1.0',
 'category_coarse': 'Food and non-alcoholic beverages',
 'category_middle': 'Food',
 'category_fine': 'Bread and cereals'}

In [49]:
len(co)

91

# ------------------------------------------ Part 5 ------------------------------------------

# Add sectors as separate activities
## 1. Choose sectors

In [ ]:
import brightway2 as bw
import pandas as pd

In [ ]:
bw.projects.set_current('GSA for ecoinvent')

In [ ]:
co = bw.Database('CH consumption 1.0')

In [ ]:
cat_option = 'category_coarse'

cat_unique = []
for act in co:
    cat_unique.append(act.get(cat_option) or 0)
cat_unique = list(set(cat_unique))

category_activities = {}
category_activities_len = {}
for cat_of_interest in cat_unique: 
    list_ = []
    for act in co:
        if act.get(cat_option) == cat_of_interest:
            list_.append(act)
    if len(list_) > 0:
        category_activities[cat_of_interest] = list_
        category_activities_len[cat_of_interest] = len(list_)

In [ ]:
dict_ = {}
for cat_of_interest, activities in category_activities.items():
        
    excs_input_ag  = []
    excs_input_ex  = []
    excs_input_ec  = []

    for act in activities:
        for exc in act.exchanges():
            if 'Agribalyse' in exc.input['database']:
                excs_input_ag.append(exc.input)
            elif 'EXIOBASE' in exc.input['database']:
                excs_input_ex.append(exc.input)
            elif 'ecoinvent 3.6 cutoff' == exc.input['database']:
                excs_input_ec.append(exc.input)
                
    dict_[cat_of_interest] = dict(
        n_activities = len(activities),
        n_agribalyse_exchanges = len(excs_input_ag), 
        n_exiobase_exchanges   = len(excs_input_ex), 
        n_ecoinvent_exchanges  = len(excs_input_ec), 
    )
        

In [ ]:
df = pd.DataFrame.from_dict(dict_).T

In [ ]:
category_activities_len

## 2. Add sectors

In [ ]:
demand_act = [act for act in co if 'average' in act['name']][0]
demand_act

In [ ]:
for cat_of_interest in category_activities:
    # Create new bw activity with a specific name
    try: co.get(cat_of_interest).delete()
    except: pass
    new_act = co.new_activity(cat_of_interest, 
                              name=cat_of_interest, 
                              location='CH', 
                              unit='1 month of consumption',
                              comment='Average consumption of one household',
                             )
    new_act.save()
    
    # Add production exchange
    new_act.new_exchange(
        input = (new_act['database'], new_act['code']),
        amount = 1,
        type = 'production'
    ).save()
    
    for exc in demand_act.exchanges():
        if exc.input.get('category_coarse')==cat_of_interest:
            new_act.new_exchange(
                input  = (exc.input['database'], exc.input['code']),
                amount = exc.amount,
                type   = 'technosphere'
            ).save()


## 3. Contribution analysis for sectors

In [ ]:
list_methods = [('IPCC 2013', 'climate change', 'GTP 100a')]
list_fus = []
for cat_of_interest in category_activities:
    list_fus.append({co.search(cat_of_interest)[0]: 1})

In [ ]:
bw.calculation_setups['sector_contribution_analysis'] = {'inv':list_fus, 'ia':list_methods}

In [ ]:
%%time
myMultiLCA = bw.MultiLCA('sector_contribution_analysis')

In [ ]:
lcia_unit = bw.Method(list_methods[0]).metadata['unit']

fus = [bw.get_activity(list(el.keys())[0])['name'][:25] for el in list_fus]
df = pd.DataFrame(index=fus, columns=[lcia_unit], data=myMultiLCA.results)
df['units'] = [bw.get_activity(list(el.keys())[0])['unit'] for el in list_fus]

In [ ]:
df.sort_values(lcia_unit, ascending=False, inplace=True)

In [ ]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    print(df)